# 欢迎来到您的第一个作业！
说明如下。请尝试一下，如果遇到困难，可以查看解决方案文件夹

# 作业练习

升级 day 1 项目，使用通过 Ollama 在本地运行的开源模型而不是 OpenAI 来总结网页

如果您不想使用付费 API，可以使用此技术进行所有后续项目。

**优点：**
1. 无 API 费用 - 开源
2. 数据不会离开您的电脑

**缺点：**
1. 性能显著低于前沿模型

## Ollama 安装回顾

只需访问 [ollama.com](https://ollama.com) 并安装即可！

安装完成后，ollama 服务器应该已经在本地运行。
如果您访问：
[http://localhost:11434/](http://localhost:11434/)

您应该会看到消息 `Ollama is running`。

如果不是，请打开一个新的终端（Mac）或 Powershell（Windows）并输入 `ollama serve`
然后在另一个终端（Mac）或 Powershell（Windows）中，输入 `ollama pull llama3.2`
然后再次尝试 [http://localhost:11434/](http://localhost:11434/)。

如果 Ollama 在您的机器上运行缓慢，请尝试使用 `llama3.2:1b` 作为替代方案。从终端或 Powershell 运行 `ollama pull llama3.2:1b`，并将下面的代码从 `MODEL = "llama3.2"` 更改为 `MODEL = "llama3.2:1b"`

In [ ]:
# imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [ ]:
# Constants

OLLAMA_API = "http://127.0.0.1:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [ ]:
# 使用与 OpenAI 相同的格式创建消息列表

messages = [
    {"role": "user", "content": "描述一下生成式 AI 的一些商业应用"}
]

In [ ]:
payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }

In [ ]:
# 让我们确保模型已加载

!ollama pull llama3.2

In [ ]:
# 如果出于任何原因这不起作用，请尝试以下单元格中的 2 个版本
# 并仔细检查本实验顶部“Ollama 安装回顾”中的说明

response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response.json()['message']['content'])

# 介绍 ollama 包

现在我们将做同样的事情，但使用优雅的 ollama python 包而不是直接的 HTTP 调用。

在底层，它与上面一样调用运行在 localhost:11434 的 ollama 服务器。

In [ ]:
import ollama
response = ollama.chat(model=MODEL, messages=messages)
print(response['message']['content'])

## 另一种方法 - 使用 OpenAI python 库连接到 Ollama

In [ ]:
# 实际上还有另一种方法，有些人可能更喜欢
# 您可以使用 OpenAI 客户端 python 库调用 Ollama：

from openai import OpenAI
ollama_via_openai = OpenAI(base_url='http://127.0.0.1:11434/v1', api_key='ollama')

response = ollama_via_openai.chat.completions.create(
    model=MODEL,
    messages=messages
)
print(response.choices[0].message.content)

## 您是否对为什么这能起作用感到困惑？

这看起来很奇怪，对吧？我们刚刚使用了 OpenAI 的代码来调用 Ollama？？这是怎么回事？！

事情是这样的：

python 类 `OpenAI` 只是 OpenAI 工程师编写的用于通过互联网调用端点的代码。

当您调用 `openai.chat.completions.create()` 时，这段 python 代码只是向以下 url 发起一个网络请求："https://api.openai.com/v1/chat/completions"

像这样的代码被称为“客户端库” - 它只是运行在您机器上的包装代码，用于发起网络请求。GPT 的实际能力运行在 OpenAI 的云端，位于此 API 之后，而不是在您的计算机上！

OpenAI 非常流行，许多其他 AI 提供商也提供了相同的网络端点，因此您可以使用相同的方法。

因此 Ollama 在您的本地设备上运行着一个端点，地址是 http://localhost:11434/v1/chat/completions
在第二周，我们将发现许多其他提供商也这样做，包括 Gemini 和 DeepSeek。

然后 OpenAI 的团队有了一个很棒的主意：他们可以扩展他们的客户端库，以便您可以指定不同的“基本 url”，并使用他们的库来调用任何兼容的 API。

就是这样！

所以当您说：`ollama_via_openai = OpenAI(base_url='http://127.0.0.1:11434/v1', api_key='ollama')`
那么这将发起相同的端点调用，但目标是 Ollama 而不是 OpenAI。

## 也尝试一下惊人的推理模型 DeepSeek

这里我们使用 DeepSeek-reasoner 的 1.5B 精简版本。
这实际上是 Qwen 的一个 1.5B 变体，已使用 Deepseek R1 生成的合成数据进行了微调。
其他大小的 DeepSeek [在这里](https://ollama.com/library/deepseek-r1)，一直到完整的 671B 参数版本，这将占用您 404GB 的硬盘空间，对于大多数人来说太大了！

In [ ]:
!ollama pull deepseek-r1:1.5b

In [ ]:
# 这可能需要几分钟才能运行！然后您应该会在 <think> 标签内看到一个引人入胜的“思考”轨迹，后面是一些不错的定义

response = ollama_via_openai.chat.completions.create(
    model="deepseek-r1:1.5b",
    messages=[{"role": "user", "content": "请给出 LLM 背后一些核心概念的定义：神经网络、注意力和 Transformer"}]
)
print(response.choices[0].message.content)

# 现在是您的练习

将 day1 的代码整合到这里，构建一个使用本地运行的 Llama 3.2 而不是 OpenAI 的网站摘要工具；使用上述任一方法。